In [ ]:
# pip install --upgrade tensorflow

In [1]:
import pandas as pd
import numpy as np
from statistics import mean
import ast
from scipy.spatial.distance import cosine, euclidean, hamming
from sklearn.preprocessing import normalize
#from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from time import time

In [35]:
"""
append user rating for selected recipe (normalized)
drop nutritions column
"""

recipe = pd.read_csv('raw-data_recipe2.csv')
recipe = recipe.drop(columns=['recipe_name'])
recipe.head()

,recipe_id,oil,flour,sugar,salt,milk,beaten,apples,meat,pepper,...,confectioners',sprinkles,available,berries,broken,bran,pectin,holes,curry,cod
0,10001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
1,90295,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
2,14685,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
3,22736,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
4,244562,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0


In [36]:
# round average rating into 2 decimal places
def avg_rate(col):
    return f'{col:.2f}'

In [37]:
recipe.shape

(22, 319)

In [38]:
# total number of unique recipe
recipe.recipe_id.nunique()

22

In [42]:
df = pd.DataFrame(recipe.drop(columns=['recipe_id']))
df.index = recipe['recipe_id']
df.head()

,oil,flour,sugar,salt,milk,beaten,apples,meat,pepper,garlic,...,confectioners',sprinkles,available,berries,broken,bran,pectin,holes,curry,cod
recipe_id,,,,,,,,,,,,,,,,,,,,,
10001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
90295,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
14685,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
22736,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
244562,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0


In [43]:
df.isnull().sum()

oil       2
flour     2
sugar     2
salt      2
milk      2
         ..
bran      2
pectin    2
holes     2
curry     2
cod       2
Length: 318, dtype: int64

In [44]:
print(df.shape)
df = df.dropna()
df.shape

(22, 318)


(20, 318)

In [48]:
# impute string data into numeric value
def text_cleaning(cols):
    if cols == '< 1':
        return 1
    else:
        return cols

In [49]:
for col in df.columns:
    df[col] = df[col].apply(text_cleaning)

In [51]:
df.head()

,oil,flour,sugar,salt,milk,beaten,apples,meat,pepper,garlic,...,confectioners',sprinkles,available,berries,broken,bran,pectin,holes,curry,cod
recipe_id,,,,,,,,,,,,,,,,,,,,,
10001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
90295,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
14685,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
22736,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
244562,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0


In [53]:
df.isna().sum()

oil       0
flour     0
sugar     0
salt      0
milk      0
         ..
bran      0
pectin    0
holes     0
curry     0
cod       0
Length: 318, dtype: int64

In [54]:
df.dtypes

oil       float64
flour     float64
sugar     float64
salt      float64
milk      float64
           ...   
bran      float64
pectin    float64
holes     float64
curry     float64
cod       float64
Length: 318, dtype: object

In [55]:
df = df.apply(pd.to_numeric)
df.dtypes

oil       float64
flour     float64
sugar     float64
salt      float64
milk      float64
           ...   
bran      float64
pectin    float64
holes     float64
curry     float64
cod       float64
Length: 318, dtype: object

In [56]:
# normalized ingredient data by columns
df_normalized = pd.DataFrame(normalize(df, axis=0))
df_normalized.columns = df.columns
df_normalized.index = df.index
df_normalized.head()

,oil,flour,sugar,salt,milk,beaten,apples,meat,pepper,garlic,...,confectioners',sprinkles,available,berries,broken,bran,pectin,holes,curry,cod
recipe_id,,,,,,,,,,,,,,,,,,,,,
10001,0.000000,0.000000,0.301511,0.000000,0.000000,0.000000,0.288675,0.000000,0.27735,0.288675,...,0.000000,0.000000,0.301511,0.288675,0.288675,0.288675,0.288675,0.00000,0.316228,0.353553
90295,0.301511,0.377964,0.000000,0.333333,0.353553,0.000000,0.288675,0.267261,0.00000,0.000000,...,0.288675,0.000000,0.000000,0.288675,0.288675,0.000000,0.288675,0.27735,0.316228,0.353553
14685,0.301511,0.000000,0.000000,0.333333,0.000000,0.333333,0.288675,0.267261,0.27735,0.288675,...,0.288675,0.000000,0.000000,0.288675,0.288675,0.000000,0.000000,0.27735,0.000000,0.000000
22736,0.000000,0.000000,0.000000,0.333333,0.000000,0.333333,0.288675,0.267261,0.00000,0.000000,...,0.000000,0.316228,0.301511,0.288675,0.288675,0.288675,0.000000,0.00000,0.316228,0.000000
244562,0.000000,0.000000,0.301511,0.000000,0.000000,0.000000,0.000000,0.267261,0.00000,0.288675,...,0.000000,0.316228,0.301511,0.288675,0.000000,0.288675,0.288675,0.00000,0.000000,0.353553


In [69]:
"""
Nutrition Recommender based on different distance calculation approaches

df_normalized: normalized nutrition data
distance_method: distance calculation approach: e.g. cosine, euclidean, hamming
recipe_id: find similar recipes based on the selected recipe
N: Top N recipe(s)

return 1) nutrition data of selected recipe and Top N recommendation, 
2) recipe id, recipe name and image of Top N recommendation
"""

def ingredient_recommender(distance_method, recipe_id, N):

    allRecipes = pd.DataFrame(df_normalized.index)
    allRecipes = allRecipes[allRecipes.recipe_id != recipe_id]
    allRecipes["distance"] = allRecipes["recipe_id"].apply(lambda x: distance_method(df_normalized.loc[recipe_id], df_normalized.loc[x]))
    TopNRecommendation = allRecipes.sort_values(["distance"]).head(N).sort_values(by=['distance', 'recipe_id'])
    # sort by distance then recipe id, the smaller value of recipe id will be picked. 
    return df_normalized.loc[recipe_id, :]
""" 
    recipe_df = recipe.set_index('recipe_id')
    recipe_id = [recipe_id]
    recipe_list = []
    image_list = []
    image_path = "./raw-data-images/{}.jpg"
    for recipeid in TopNRecommendation.recipe_id:
        recipe_id.append(recipeid)   # list of recipe id of selected recipe and recommended recipe(s)
        recipe_list.append("{}  {}".format(recipeid, recipe_df.at[recipeid, 'recipe_name']))
        image_list.append(image_path.format(recipeid))
    
    image_array = []
    for imagepath in image_list:
        img = image.load_img(imagepath)
        img = image.img_to_array(img, dtype='int')
        image_array.append(img)
        
    fig = plt.figure(figsize=(15,15))
    gs1 = gridspec.GridSpec(1, N)
    axs = []
    for x in range(N):
        axs.append(fig.add_subplot(gs1[x]))
        axs[-1].imshow(image_array[x])
    [axi.set_axis_off() for axi in axs]
    for axi, x in zip(axs, recipe_list):
        axi.set_title(x)
"""
    

' \n    recipe_df = recipe.set_index(\'recipe_id\')\n    recipe_id = [recipe_id]\n    recipe_list = []\n    image_list = []\n    image_path = "./raw-data-images/{}.jpg"\n    for recipeid in TopNRecommendation.recipe_id:\n        recipe_id.append(recipeid)   # list of recipe id of selected recipe and recommended recipe(s)\n        recipe_list.append("{}  {}".format(recipeid, recipe_df.at[recipeid, \'recipe_name\']))\n        image_list.append(image_path.format(recipeid))\n    \n    image_array = []\n    for imagepath in image_list:\n        img = image.load_img(imagepath)\n        img = image.img_to_array(img, dtype=\'int\')\n        image_array.append(img)\n        \n    fig = plt.figure(figsize=(15,15))\n    gs1 = gridspec.GridSpec(1, N)\n    axs = []\n    for x in range(N):\n        axs.append(fig.add_subplot(gs1[x]))\n        axs[-1].imshow(image_array[x])\n    [axi.set_axis_off() for axi in axs]\n    for axi, x in zip(axs, recipe_list):\n        axi.set_title(x)\n'

In [70]:
ingredient_recommender(cosine, 10001, 3)

oil       0.000000
flour     0.000000
sugar     0.301511
salt      0.000000
milk      0.000000
            ...   
bran      0.288675
pectin    0.288675
holes     0.000000
curry     0.316228
cod       0.353553
Name: 10001, Length: 318, dtype: float64